In [2]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import numpy.linalg as la

In [48]:
# 1.1 - 1.3

seed = 1

X_mean = np.array([0]*10)
X_1 = np.array([[0.2]*10]*10)
X_2 = np.identity(10)*0.8
X_cov = X_1 + X_2

X_distr = scipy.stats.multivariate_normal(cov = X_cov, mean = X_mean, seed = seed)

B = np.asarray([[0.12], [0.16], [0], [0], [0], [0], [0], [0], [0], [0]])

error_mean = 0
error_var = 1
error_distr = scipy.stats.norm(loc = error_mean, scale = np.sqrt(error_var))

sample_betas = []
reg_lst = []
for s in range(5000):
    X = X_distr.rvs(size=1000)
    error = np.vstack(error_distr.rvs(size=1000))
    y = (X @ B) + error
    best_beta = None
    best_reg = None
    best_R2 = 0
    for i in range(0,10):
        for j in range(i+1,10):
            X_sub = X[:, [i,j]]
            sample_beta = la.inv(X_sub.T @ X_sub) @ X_sub.T @ y
            y_predicted = (X_sub @ sample_beta)
            R2 = scipy.stats.pearsonr(y.flatten(), y_predicted.flatten())[0]
            if R2 > best_R2:
                best_R2 = R2
                best_beta = sample_beta
                best_reg = [i + 1, j + 1]
    sample_betas.append(best_beta.flatten())
    reg_lst.append(best_reg)
beta_array = np.vstack(sample_betas)
print(beta_array.mean(axis=0))
print(beta_array.var(axis=0))
x = [0 for f in reg_lst if f == [1, 2]]
print(len(x)/5000)
            
            
                

[0.12610351 0.15369894]
[0.00102121 0.00177799]
0.9434


In [50]:
# 1.4 - 1.5


# From TA session:

#Coordinate Descent 
maxiter = 100
num_lambda = 200
sum_of_x = 1000
p = 10

lambda_vec = np.linspace(0.001,1,num_lambda)

X = X_distr.rvs(size=1000)
error = np.vstack(error_distr.rvs(size=1000))
y = (X @ B) + error

#Results Holder
beta_cd = np.zeros((num_lambda,p))

for l in range(len(lambda_vec)):
    b_lasso = np.zeros((p,1))
    res = y - X@b_lasso
    for s in range(maxiter):
        for v in range(p):
            #Create Y - \sum_{not v} X_i b
            res = res + X[:,v].reshape(-1,1)*b_lasso[v]

            #update b lasso 
            b_ols = (X[:,v]@res)/sum_of_x
            shrinkage = np.abs(b_ols) - lambda_vec[l]
            b_lasso[v] = np.sign(b_ols)*np.maximum(0,shrinkage)

            #Update residual
            res = res - X[:,v].reshape(-1,1)*b_lasso[v]
    beta_cd[l,:] = b_lasso.T
print(beta_cd)

[[ 0.11567049  0.09762534 -0.01791362 ... -0.04090739 -0.01678223
   0.03917239]
 [ 0.10907403  0.09071247 -0.01155356 ... -0.03402461 -0.00992451
   0.03321934]
 [ 0.10233429  0.08359016 -0.00544853 ... -0.02752978 -0.00329253
   0.02713608]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [54]:
# 1.6


# From TA session:

#Coordinate Descent
num_lambda = 200
maxiter = 100
sum_of_x = 1000
p = 10

lambda_vec = np.linspace(0.001,1,num_lambda)

correct_count = 0

for s in range(5000):
    X = X_distr.rvs(size=1000)
    error = np.vstack(error_distr.rvs(size=1000))
    y = (X @ B) + error
    lambda_vec = 0.001
    b_lasso = np.ones((p,1))
    while np.count_nonzero(b_lasso.T) > 2:
        b_lasso = np.zeros((p,1))
        res = y - X@b_lasso
        for s in range(maxiter):
            for v in range(p):
                #Create Y - \sum_{not v} X_i b
                res = res + X[:,v].reshape(-1,1)*b_lasso[v]

                #update b lasso 
                b_ols = (X[:,v]@res)/sum_of_x
                shrinkage = np.abs(b_ols) - lambda_vec
                b_lasso[v] = np.sign(b_ols)*np.maximum(0,shrinkage)

                #Update residual
                res = res - X[:,v].reshape(-1,1)*b_lasso[v]
            lambda_vec += 0.001
    if (np.abs(b_lasso.flatten()[0]) != 0) and (np.abs(b_lasso.flatten()[1]) != 0):
        correct_count += 1
print(correct_count / 5000)

0.7636


In [59]:
# 2.1

seed = 1

X_mean = np.array([0]*100)
X_1 = np.array([[0.2]*100]*100)
X_2 = np.identity(100)*0.8
X_cov = X_1 + X_2

X_distr = scipy.stats.multivariate_normal(cov = X_cov, mean = X_mean, seed = seed)

B_lst = [0.3]*5 + [0]*95
B = np.asarray(B_lst).T

error_distr = scipy.stats.logistic()

sample_betas = []
reg_lst = []
for s in range(5000):
    X = X_distr.rvs(size=1000)
    error = np.vstack(error_distr.rvs(size=1000))
    indic_arr = (X @ B + error).flatten()
    y_lst = [np.sign(x) for x in indic_arr]
    print(y_lst)

    '''
    best_beta = None
    best_reg = None
    best_R2 = 0
    for i in range(0,10):
        for j in range(i+1,10):
            X_sub = X[:, [i,j]]
            sample_beta = la.inv(X_sub.T @ X_sub) @ X_sub.T @ y
            y_predicted = (X_sub @ sample_beta)
            R2 = scipy.stats.pearsonr(y.flatten(), y_predicted.flatten())[0]
            if R2 > best_R2:
                best_R2 = R2
                best_beta = sample_beta
                best_reg = [i + 1, j + 1]
    sample_betas.append(best_beta.flatten())
    reg_lst.append(best_reg)
beta_array = np.vstack(sample_betas)
print(beta_array.mean(axis=0))
print(beta_array.var(axis=0))
x = [0 for f in reg_lst if f == [1, 2]]
print(len(x)/5000)
'''
print(y)

KeyboardInterrupt: 